## Dependencies

In [ ]:
!pip install weaviate-client

## Connect to Weaviate

In [ ]:
import weaviate, os

# Connect to your local Weaviate instance deployed with Docker
client = weaviate.connect_to_local(
  headers={
    "X-AWS-Access-Key": os.getenv("AWS_ACCESS_KEY"), # Replace with your AWS access key - recommended: use env var
    "X-AWS-Secret-Key": os.getenv("AWS_SECRET_KEY"), # Replace with your AWS secret key - recommended: use env var
  }
)

# Option 2
# Connect to your Weaviate Client Service cluster
# client = weaviate.connect_to_wcs(
#     cluster_url="WCS-CLUSTER-ID",                             # Replace with your WCS cluster ID
#     auth_credentials=weaviate.auth.AuthApiKey("WCS-API-KEY"), # Replace with your WCS API KEY - recommended: use env var
#     headers={
#         "X-AWS-Access-Key": os.getenv("AWS_ACCESS_KEY"), # Replace with your AWS access key - recommended: use env var
#         "X-AWS-Secret-Key": os.getenv("AWS_SECRET_KEY"), # Replace with your AWS secret key - recommended: use env var
#     }
# )

client.is_ready()

## Create a collection
> Collection stores your data and vector embeddings.

In [ ]:
# Note: in practice, you shouldn"t rerun this cell, as it deletes your data
# in "JeopardyQuestion", and then you need to re-import it again.
import weaviate.classes.config as wc

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_aws(
        service="bedrock",   #this is crucial
        model="cohere.embed-english-v3", # select the model, make sure it is enabled for your account
        # model="amazon.titan-embed-text-v1", # select the model, make sure it is enabled for your account
        region="eu-west-2"               # select your region
    ),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="Question", data_type=wc.DataType.TEXT), 
        wc.Property(name="Answer", data_type=wc.DataType.TEXT),
        wc.Property(name="Category", data_type=wc.DataType.TEXT, skip_vectorization=True), 
    ]
)

print("Successfully created collection: JeopardyQuestion.")

## Import the Data

In [ ]:
import requests, json
url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

# Insert data objects
response = jeopardy.data.insert_many(data)

# Note, the `data` array contains 10 objects, which is great to call insert_many with.
# However, if you have a milion objects to insert, then you should spit them into smaller batches (i.e. 100-1000 per insert)

if (response.has_errors):
    print(response.errors)
else:
    print("Insert complete.")

## Hybrid Search

The `alpha` parameter determines the weight given to the sparse and dense search methods. `alpha = 0` is pure sparse (bm25) search, whereas `alpha = 1` is pure dense (vector) search. 

Alpha is an optional parameter. The default is set to `0.75`.

### Hybrid Search only

The below query is finding Jeopardy questions about animals and is limiting the output to only two results. Notice `alpha` is set to `0.80`, which means it is weighing the vector search results more than bm25. If you were to set `alpha = 0.25`, you would get different results. 

In [ ]:
# note, you can reuse the collection object from the previous cell.
# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

response = jeopardy.query.hybrid(
    query="northern beast",
    alpha=0.8,
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

### Hybrid Search on a specific property

The `properties` parameter allows you to list the properties that you want bm25 to search on.

In [ ]:
response = jeopardy.query.hybrid(
    query="northern beast",
    query_properties=["question"],
    alpha=0.8,
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

### Hybrid Search with a `where` filter

Find Jeopardy questions about elephants, where the category is set to Animals.

In [ ]:
import weaviate.classes.query as wq # wq is an alias to save us from typing weaviate.classes everywhere ;)

response = jeopardy.query.hybrid(
    query="northern beast",
    alpha=0.8,
    filters=wq.Filter.by_property("category").equal("Animals"),
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

### Hybrid Search with a custom vector

You can pass in your own vector as input into the hybrid query, by using the `vector` parameter. 

In [25]:
vector = [0.028259277, -0.005809784, -0.005832672, 0.0037231445, -0.042633057, -0.047668457, 0.009521484, -0.02571106, 0.0009598732, -0.0010766983, -0.01600647, -0.0017185211, 0.009780884, -0.026641846, 0.027923584, -0.04244995, 0.025848389, 0.010643005, 0.029571533, 0.0019989014, 0.014778137, 0.017547607, -0.013549805, -0.101257324, 0.025527954, -0.00045132637, 0.0031261444, -0.012763977, -0.026107788, -0.03012085, 0.024475098, 0.019592285, 0.016052246, 0.05319214, -0.024215698, 0.076049805, -0.02406311, 0.011192322, 0.019485474, 0.026748657, 0.052124023, -0.036834717, -0.0049362183, -0.02456665, -0.027664185, -0.005207062, 0.048187256, -0.012329102, 0.044006348, -0.015655518, 0.021194458, 0.05822754, 0.023117065, -0.0099105835, -0.018600464, 0.03942871, -0.01977539, -0.015350342, 0.03970337, -0.058898926, -0.019622803, -0.019210815, 0.059387207, 0.05041504, 0.004573822, -0.005367279, 0.0154418945, 0.018096924, 0.0034923553, -0.03314209, -0.0030784607, -0.010902405, 0.005180359, -0.050476074, 0.008117676, 0.008811951, -0.013420105, -0.006340027, 0.019073486, -0.012481689, 0.020996094, -0.021835327, -0.012260437, 0.011276245, -0.012962341, 0.019134521, -0.0018033981, -0.033966064, 0.035827637, -0.035583496, 7.7843666e-05, 0.09887695, 0.054748535, -0.03062439, -0.080566406, -0.029632568, -0.04586792, -0.0592041, 0.01676941, 0.032592773, -0.00869751, -0.023895264, 0.022018433, -0.015823364, -0.02381897, 0.008277893, 0.013252258, 0.013626099, -0.009086609, 0.015319824, -0.046539307, 0.02053833, -0.009552002, -0.031280518, -0.029953003, 0.039367676, 0.0049591064, 0.011047363, 0.00724411, 0.04272461, 0.034362793, 0.072387695, 0.029312134, -0.05343628, -0.001162529, -0.00026154518, -0.011695862, -0.059936523, -0.00086069107, -0.029724121, -0.040161133, 0.046722412, 0.012107849, -0.007610321, -0.018966675, -0.012748718, -0.034820557, -0.041931152, -0.004501343, 0.048858643, -0.018432617, 0.0012578964, 0.10235596, -0.0032424927, -0.028961182, 0.04067993, 0.032409668, -0.023986816, -0.010749817, 0.028778076, -0.07476807, 0.011711121, 0.017913818, 0.03274536, 0.009628296, -0.02142334, 0.008346558, -0.026519775, 0.057495117, -0.017730713, -0.013725281, 0.052825928, 0.100097656, -0.026275635, -0.044158936, -0.0836792, -0.04638672, -0.005470276, 0.033447266, 0.05078125, 0.0030231476, 0.010887146, 0.076049805, 0.064941406, -0.0022258759, 0.010269165, -0.029891968, 0.014381409, 0.058807373, -0.035095215, -0.03451538, -0.042053223, 0.015899658, 0.020233154, 0.0287323, -0.074645996, 0.02848816, -0.02420044, -0.0066947937, 0.03643799, 0.017120361, 0.0012817383, -0.032928467, 0.009925842, -0.010231018, -0.003458023, -0.016738892, 0.024612427, -0.011894226, -0.013046265, 0.011894226, 0.021896362, 0.013046265, -0.020095825, 0.004180908, -0.03262329, 0.0046043396, 0.009895325, -0.006511688, 0.030227661, 0.02015686, -0.004776001, -0.045959473, 0.0043525696, 0.01626587, 0.030303955, -0.017349243, -0.005836487, -0.024658203, 0.049591064, -0.046844482, 0.014968872, -0.07720947, 0.02330017, 0.0012454987, 0.0032310486, -0.0070762634, 0.018127441, -0.039489746, 0.064086914, -0.047210693, -0.03805542, -0.023239136, -0.028839111, -0.015625, 0.022613525, 0.039001465, -0.0012359619, 0.021408081, 0.0044937134, 0.030715942, -0.011169434, 0.0028495789, -0.0002565384, 0.016311646, -0.019821167, 0.03540039, -0.039215088, -0.033813477, -0.008918762, 0.05734253, -0.037506104, 0.022659302, -0.007820129, -0.01739502, 0.036590576, 0.019836426, 0.05307007, -0.036193848, 0.025680542, -0.023376465, 0.016052246, 0.0018262863, -0.012252808, -0.04876709, 0.017974854, -0.014228821, 0.015556335, -0.033843994, -0.0016307831, 0.033691406, 0.021774292, -0.010032654, 0.025650024, 0.059539795, 0.054595947, -0.05267334, -0.00019991398, 0.005760193, 0.031036377, -0.021743774, 0.024841309, 0.02671814, -0.012336731, 0.014122009, 0.029296875, 0.016540527, -0.03302002, -0.05429077, 0.011520386, 0.061706543, 0.01637268, -0.025604248, -0.009803772, -0.02468872, 0.012893677, 0.039001465, 0.053527832, -0.022369385, -0.018585205, -0.017669678, 0.0073623657, -0.008628845, -0.040405273, 0.012176514, -0.012702942, 0.035217285, 0.016403198, 0.022338867, 0.0062561035, -0.06463623, 0.024780273, 0.007888794, -0.055145264, 0.0030765533, -0.015098572, -0.03277588, -0.03692627, -0.09399414, -0.031982422, -0.028884888, -0.005809784, -0.015991211, -0.02166748, -0.020812988, 0.023010254, -0.03933716, 0.007347107, -0.060272217, 0.023529053, 0.031585693, -0.012435913, -0.009597778, -0.022216797, -0.023010254, 0.013511658, 0.026794434, 0.03945923, 0.02406311, -0.0021820068, 0.064819336, -0.01876831, 0.005592346, 0.033416748, -0.0042266846, -0.00548172, -0.027435303, 0.009391785, 0.034423828, -0.005168915, -0.01083374, -0.04840088, -0.034118652, -0.007881165, 0.0074539185, -0.0062942505, 0.039520264, -0.0015716553, 0.04321289, -0.057891846, 0.007926941, -0.037261963, -0.049560547, 0.02078247, 0.022109985, -0.026443481, -0.0054359436, -0.011001587, 0.018173218, 0.010253906, -0.0008006096, 0.009216309, -0.048461914, -0.0048217773, -0.020767212, 0.0024814606, 0.045532227, -0.05343628, 0.029205322, 0.016036987, 0.010108948, 0.011138916, -0.014984131, -0.03213501, 0.06100464, 0.02848816, -0.012329102, 0.015731812, 0.043304443, 0.009719849, -0.011184692, 0.0071144104, -0.033294678, -0.036834717, -0.012481689, -0.0011739731, -0.0110321045, 0.016204834, -0.03515625, 0.0056610107, 0.063964844, 0.012924194, -0.0007157326, 0.010467529, -0.054382324, 0.036834717, 0.0028972626, 0.022247314, 0.0119018555, 0.0413208, -0.03540039, 0.009170532, -0.023529053, -0.019256592, -0.018493652, 0.011039734, 0.09173584, 0.017150879, 0.014160156, -0.06210327, -0.017593384, 0.041870117, -0.01348114, -0.011260986, -0.021331787, 0.025161743, -0.031921387, -0.015037537, 0.01411438, 0.029205322, -0.024276733, -0.0018320084, -0.014671326, -0.020187378, 0.025405884, 0.054016113, -0.052368164, 0.021713257, -0.025222778, 0.0070610046, 0.010864258, 0.0009713173, -0.0084991455, -0.03201294, -0.024261475, -0.040740967, 0.035339355, 0.03768921, -0.009941101, -0.0036144257, -0.03842163, 0.015823364, 0.029907227, 0.037506104, -0.031677246, 0.037078857, 0.053009033, 0.029144287, 0.029403687, -0.0027370453, -0.008300781, 0.0072135925, -0.07501221, -0.0010471344, -0.012161255, -0.015403748, 0.010658264, 0.02911377, 0.0050621033, -0.025527954, 0.0038394928, 0.041778564, -0.0007138252, -0.0043296814, 0.041900635, 0.040618896, -0.081848145, -0.023971558, -0.011245728, 0.05709839, 0.018432617, -0.002538681, 0.033477783, 0.054504395, -0.00869751, -0.010063171, -0.029815674, -0.036865234, 0.043670654, -0.013137817, -0.0065841675, -0.020050049, 0.027130127, -0.0013074875, -0.017471313, 0.00021922588, 0.027114868, -0.05432129, 0.0056419373, -0.029220581, -0.062042236, 0.019073486, 0.04840088, -0.009552002, 0.021697998, -0.008346558, -0.015014648, 0.021377563, 0.05267334, 0.07531738, -0.03878784, 0.003458023, 0.026931763, 0.014251709, -0.02015686, 0.049041748, 0.023254395, 0.003522873, -0.014045715, -0.026687622, 0.033599854, -0.013267517, 0.10095215, 0.038879395, -0.040161133, -0.018585205, 0.067993164, -0.04660034, -0.0154800415, -0.028045654, 0.01083374, 0.010765076, 0.012763977, 0.016723633, 0.010307312, 0.027114868, -0.032806396, 0.0045547485, -0.032928467, -0.030181885, -0.060272217, -0.010063171, 0.041412354, 0.003774643, 0.0029735565, -0.010795593, -0.013511658, 0.029144287, -0.01398468, 0.034362793, 0.0012788773, -0.019638062, -0.0143966675, 0.0032978058, 0.018051147, -0.0049934387, -0.08074951, 0.01751709, -0.016677856, 0.004497528, 0.016555786, 0.017822266, -0.030822754, -0.0009326935, -0.008857727, 0.013587952, 0.030975342, -0.04559326, 0.017211914, -0.00013267994, 0.0013751984, -0.023239136, -0.00095796585, -0.017684937, -0.02116394, -0.015083313, -0.025421143, 0.029312134, 0.036712646, 0.103393555, 0.024230957, 0.0071411133, -0.0892334, 0.017532349, -0.011375427, 0.019348145, -0.005455017, -0.053588867, -0.037322998, -0.041748047, 0.016189575, 0.0031433105, -0.014846802, -0.081726074, -0.00046300888, -0.013946533, -6.699562e-05, 0.04763794, 0.03982544, -0.0017995834, -0.030792236, 0.0011920929, 0.012466431, -0.051971436, 0.031433105, -0.023849487, 0.031143188, -0.017044067, -0.0023727417, 0.0005850792, -0.007801056, -0.02557373, -0.023452759, 0.00806427, -0.03161621, 0.02609253, 0.03253174, 0.0063171387, 0.036712646, -0.039093018, -0.011985779, 0.017654419, 0.0073280334, 0.0018901825, 0.020233154, 0.0140686035, -0.024353027, -0.016021729, -0.023513794, 0.012382507, -0.025527954, -0.022109985, -0.033325195, -0.064208984, 0.0024089813, 0.06976318, -0.005542755, -0.01965332, 0.0011959076, 0.051971436, 0.028533936, 0.010406494, 0.009788513, -0.05429077, 0.009384155, -0.015655518, -0.008262634, 0.016342163, -0.007232666, 0.0345459, 0.010429382, 3.9756298e-05, 0.0073547363, -0.025299072, 0.012588501, -0.001241684, -0.0034427643, 0.06488037, -0.01928711, -0.0010728836, -0.005420685, 0.048675537, -0.034820557, -0.0051078796, 0.045440674, 0.027938843, -0.004043579, 0.026916504, -0.0021190643, 0.010528564, 0.0066833496, -0.047668457, -0.013633728, -0.026473999, -0.021438599, -0.0005273819, 0.00059366226, 0.017410278, -0.027313232, 0.0037708282, -0.024246216, -0.076293945, 0.0060272217, 0.015571594, -0.01953125, 0.03390503, 0.019897461, 0.015838623, -0.023544312, -0.0066184998, -0.054534912, -0.03756714, -0.019561768, -0.044647217, 0.0021533966, -0.027267456, -0.03942871, -0.024551392, -0.017410278, 0.026626587, -0.0055160522, -0.019439697, 0.03643799, -0.03555298, 0.0039043427, 0.030792236, -0.028381348, 0.028747559, -0.022979736, -0.011726379, 0.012931824, 0.06274414, -0.05432129, -0.025970459, 0.0146865845, -0.00091838837, 0.022445679, -0.007003784, 0.008926392, 0.023742676, -0.02067566, -0.058380127, 0.0418396, -0.015449524, 0.036254883, -0.004951477, -0.00749588, -0.016464233, 0.0043525696, -0.0027256012, 0.040527344, 0.005683899, -0.0019340515, 0.0099487305, 0.0021743774, -0.03286743, 0.00409317, 0.029541016, -0.011695862, 0.009292603, -0.017425537, -0.032470703, 0.025604248, -0.009231567, -0.006504059, 0.037902832, 0.01953125, 0.049957275, 0.009773254, 0.0134887695, 0.04486084, -0.014320374, -0.0026988983, -0.031311035, 0.019729614, -0.041809082, 0.011856079, 0.011299133, 0.005168915, -0.030075073, -0.027297974, 0.016342163, 0.016998291, 0.008865356, 0.021408081, 0.0029735565, 0.011581421, 0.02420044, -0.0010890961, 0.013336182, 0.029006958, -0.0020523071, 0.0015878677, -0.008842468, -0.0061035156, 0.03869629, 0.007888794, 0.028869629, -0.029937744, -0.002779007, -0.008964539, 0.030914307, -0.050323486, 0.003019333, 0.0022583008, 0.01272583, 0.0690918, -0.010772705, 0.049835205, 0.039001465, -0.009315491, 0.0044670105, 0.00052690506, -0.015235901, 0.013793945, 0.009986877, -0.01146698, -0.013763428, 0.009300232, -0.03744507, -0.013687134, -0.06536865, 0.03781128, 0.056243896, 0.018981934, 0.0054740906, -0.0045166016, 0.064697266, 0.028640747, 0.062805176, 0.0055770874, 0.026138306, -0.027389526, -0.016860962, -0.027862549, 0.04800415, -0.023498535, 0.0059280396, 0.019989014, -0.111572266, 0.018005371, 0.06573486, -0.007980347, 0.0032520294, -0.023529053, 0.039154053, 0.024032593, -0.04360962, -0.035705566, -0.02078247, 0.048675537, 0.0012683868, -0.02041626, 0.037750244, 0.021759033, -0.041656494, -0.0067596436, 0.0013818741, 0.06335449, 0.0045166016, -0.007083893, 0.008544922, -0.04071045, -0.027191162, 0.011062622, -0.058166504, 0.017074585, 0.010551453, 0.06793213, 0.060943604, 0.017349243, 0.012992859, -0.015258789, -0.011756897, 0.02609253, 0.06866455, 0.035064697, -0.02267456, -0.020996094, -0.019821167, 0.0049057007, 0.04800415, 0.03390503, -0.014160156, -0.007896423, 0.0052948, -0.057556152, 0.012290955, -0.043762207, 0.04067993, 0.01361084, 0.0501709, 0.016784668, -0.057891846, -0.047821045, 0.0044898987, -0.00023519993, -0.026168823, 0.009468079, 0.03439331, -0.035217285, -0.028076172, 0.0140686035, -0.031036377, -0.012825012, -0.05657959, 0.031585693, -0.057769775, 0.018585205, 0.00010883808, 0.0309906, -0.0385437, -0.019973755, 0.031829834, -0.00055885315, 0.04309082, -0.018127441, 0.014778137, -0.0335083, -0.0063591003, -0.0032901764, 0.000582695, 0.02671814, -0.035461426, -0.066101074, 0.04220581, 0.062927246, -0.013450623, -0.026138306, -0.032806396, -0.04058838, -0.019012451, -0.03753662, -0.0027332306, -0.0009050369, -0.005382538, -0.09802246, -0.03353882, -0.01725769, 0.043151855, -0.018981934, 0.017715454, 0.018920898, -0.019042969, -0.008666992, 0.030639648, 0.02720642, -0.04812622, -0.0050811768, -0.0068244934, -0.036102295, -0.024383545, -0.014511108, -0.060546875, 0.02659607, -0.032104492, 0.023361206, -0.0034313202, -0.00065660477, -0.05758667, -0.018310547, 0.010360718, 0.015586853, -0.0031738281, -0.05480957, 0.029968262, 0.0021896362, -0.028198242, -0.058624268, -0.032470703, -0.017868042, -0.025283813, -0.05871582, 0.029464722, -0.019592285, 0.053222656, -0.020492554, -0.029541016, 0.021713257, 0.016723633, -0.030059814, 0.00920105, -0.013145447, -0.001660347, 0.029251099, 0.020111084, -0.006839752, -0.007835388, -0.088134766, -0.01424408, 0.014945984, -0.04876709, 0.0077285767, -0.09887695, -0.026153564, -0.0135269165, 0.030883789, 0.037628174, -0.00092315674, 0.05126953, -0.033813477, 0.097839355, -0.014503479, -0.015403748, -0.019042969, 0.04434204, -0.012207031, 0.027694702, 0.008506775, 0.005290985, -0.0012292862, 0.06185913, 0.012962341, 0.013313293, -0.00881958, -0.00010293722, 0.02947998, 0.02381897, -0.0025615692, -0.017028809, -0.038757324, 0.010108948, 0.029312134, 0.003791809, -0.026977539, 0.04537964, 0.04824829, -0.016693115, -0.0014781952, 0.006843567, -0.007926941, 0.012184143, -0.030944824, 0.0013780594, -0.020233154, -0.030792236, -0.0036201477, 0.028167725, 0.02645874, -0.027679443, -0.012878418, -0.033996582, 0.009506226, 0.014678955]

response = jeopardy.query.hybrid(
    query="animal",
    vector=vector,
    limit=2
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

ID: 321a36ce-693b-402f-89c6-efdd3344d982
Data: {
  "answer": "Antelope",
  "question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
  "category": "ANIMALS"
} 

ID: d471e1e0-2420-417b-9256-594a7b4b5817
Data: {
  "answer": "Elephant",
  "question": "It's the only living mammal in the order Proboseidea",
  "category": "ANIMALS"
} 

